In [1]:
import tensorflow as tf
import numpy as np
import cv2
from random import randint
from tqdm import tqdm

In [2]:
img_size = 256
# Placeholders for input data and the targets
X_input = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Input')
X_input_mask = tf.placeholder(tf.float32, (None, img_size, img_size, 1), name='Input_bw')
X_target = tf.placeholder(tf.float32, (None, img_size, img_size, 3), name='Target')

In [3]:
def encoder(z, reuse=False):
    """
    Encode part of the autoencoder.
    :param x: input to the autoencoder
    :param reuse: True -> Reuse the encoder variables, False -> Create or search of variables before creating
    :return: tensor which is the hidden latent variable of the autoencoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Encoder'):
        filters = {1:3,2:3,3:3}
        conv1 = tf.layers.conv2d(z, filters[1], (3,3),strides=(2, 2), padding='valid',activation=tf.nn.relu, use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        conv35 = tf.layers.conv2d(conv1, filters[2], (5,5),strides=(1,1),padding='valid',activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        #conv_shortcut = conv
        conv5 = tf.layers.conv2d(conv35, filters[3],(5,5), strides=(2, 2), padding='valid', activation=tf.nn.relu,use_bias=True, kernel_initializer='random_uniform', bias_initializer=tf.zeros_initializer())
        print('conv1',np.shape(conv1))
#         print('conv2',np.shape(conv2))
        print('conv3',np.shape(conv35))
#         print('conv4',np.shape(conv4))
        print('conv5',np.shape(conv5))
        lv = tf.contrib.layers.flatten(conv5)
    return conv5, lv

In [4]:
# conv, lv = encoder(X_input)

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `layer.__call__` method instead.
conv1 (?, 127, 127, 3)
conv3 (?, 123, 123, 3)
conv5 (?, 60, 60, 3)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.


In [5]:
class vgg16:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs
        self.convlayers()
#         self.fc_layers()
        self.map = self.pool5
#         self.probs = tf.nn.softmax(self.fc3l)
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            images = self.imgs-mean

        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv1_1.shape)

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv1_2.shape)
        # pool1
        self.pool1 = tf.nn.max_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv2_1.shape)
        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv2_2.shape)
        # pool2
        self.pool2 = tf.nn.max_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv3_1.shape)
        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv3_2.shape)
        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv3_3.shape)
        # pool3
        self.pool3 = tf.nn.max_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv4_1.shape, "conv4_1")
        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv4_2.shape, "conv4_2")
        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv4_3.shape, "conv4_3")
        # pool4
        self.pool4 = tf.nn.max_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv5_1.shape, "conv5_1")

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv5_2.shape, "conv5_2")
        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights')
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=True, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]
            print(self.conv5_3.shape, "conv5_3")
        # pool5
        self.pool5 = tf.nn.max_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

#     def fc_layers(self):
#         # fc1
#         with tf.name_scope('fc1') as scope:
#             shape = int(np.prod(self.pool5.get_shape()[1:]))
#             fc1w = tf.Variable(tf.truncated_normal([shape, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc1b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             pool5_flat = tf.reshape(self.pool5, [-1, shape])
#             fc1l = tf.nn.bias_add(tf.matmul(pool5_flat, fc1w), fc1b)
#             self.fc1 = tf.nn.relu(fc1l)
#             self.parameters += [fc1w, fc1b]

#         # fc2
#         with tf.name_scope('fc2') as scope:
#             fc2w = tf.Variable(tf.truncated_normal([4096, 4096],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc2b = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             fc2l = tf.nn.bias_add(tf.matmul(self.fc1, fc2w), fc2b)
#             self.fc2 = tf.nn.relu(fc2l)
#             self.parameters += [fc2w, fc2b]

#         # fc3
#         with tf.name_scope('fc3') as scope:
#             fc3w = tf.Variable(tf.truncated_normal([4096, 1000],
#                                                          dtype=tf.float32,
#                                                          stddev=1e-1), name='weights')
#             fc3b = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),
#                                  trainable=True, name='biases')
#             self.fc3l = tf.nn.bias_add(tf.matmul(self.fc2, fc3w), fc3b)
#             self.parameters += [fc3w, fc3b]

    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            print(i, k, np.shape(weights[k]))
            sess.run(self.parameters[i].assign(weights[k]))

In [6]:
def decode(x):
    act = tf.nn.relu(x)
    upsample1 = tf.image.resize_nearest_neighbor(act, (100,100))
    conv5 = tf.layers.conv2d(upsample1, 40, (7,7), padding='same',activation=tf.nn.relu)
    upsample2 = tf.image.resize_nearest_neighbor(conv5, (180,180))
    conv6 = tf.layers.conv2d(upsample2, 20, (5,5), padding='same',activation=tf.nn.relu)
    upsample3 = tf.image.resize_nearest_neighbor(conv6, (256,256))
    conv7 = tf.layers.conv2d(upsample3, 3, (3,3), padding='same',activation=tf.nn.relu)
    print('conv7', np.shape(conv7))
    return conv7

In [7]:
out = decode(conv)

conv7 (?, 256, 256, 3)


In [8]:
def load_dataset(i):
    img_shape=256
    training_label = []
    for j in range((i*62),(62+(i*62))):
        image = cv2.imread('D:\Temp\data_256\h\highway/00000%d.jpg' %(j+100))
        training_label.append(image)
                
    return training_label, training_label

In [9]:
if __name__ == '__main__':
#     sess = tf.Session()
    vgg = vgg16(X_input, 'vgg17_weights.npz', None)
    conv, ld = encoder(vgg.pool5)
#     latent_vector = tf.contrib.layers.flatten(vgg.pool5)
    reconstructed_image = decode(conv)
    
    lr=0.001
    L2_loss = tf.reduce_mean(tf.square(X_target - reconstructed_image))
    reconstruction_optimizer = tf.train.AdamOptimizer(learning_rate=lr,
                                                       beta1=0.9).minimize(L2_loss)
    
    batch_size=62
    epochs=100
    
    samples_n=[] 
    minibatch_loss_n = 0
    cost = []
    DATA_PATH = "/content/gdrive/My Drive/Data/"
    saver = tf.train.Saver()
    latentvarlist = []
    minibatch_loss = 0
    with tf.Session() as sess:
        init=tf.global_variables_initializer()
        sess.run(init)
        for epoch in range(epochs):            
            num_batches=170
            temp=0
            for i in tqdm(range(1)):
                X, Y = load_dataset(i)
                _, temp_loss = sess.run([reconstruction_optimizer, L2_loss], feed_dict={X_input:X, X_target:Y})
                o = sess.run(latent_vector, feed_dict={X_input:X})
                f= open("%d.txt" %(i),"w+")
                f.write(o)
                f.close() 
                tmp = temp_loss /(int(len(filesPath)/100*75)) 
                total_loss += tmp
                total_acc = train_accuracy + total_acc

            total_loss = total_loss/(int(len(filesPath)/100*75))
            total_acc = total_acc/(int(len(filesPath)/100*75))

            print ("Cost and Accuracy after epoch %i : %f, acc : %f " % (epoch, total_loss, total_acc))
            cost.append(total_loss)
        


conv1 (?, 127, 127, 3)
conv3 (?, 123, 123, 3)
conv5 (?, 60, 60, 3)
(?, 60, 60, 64)
(?, 60, 60, 64)
(?, 30, 30, 128)
(?, 30, 30, 128)
(?, 15, 15, 256)
(?, 15, 15, 256)
(?, 15, 15, 256)
(?, 8, 8, 512) conv4_1
(?, 8, 8, 512) conv4_2
(?, 8, 8, 512) conv4_3
(?, 4, 4, 512) conv5_1
(?, 4, 4, 512) conv5_2
(?, 4, 4, 512) conv5_3
conv7 (?, 256, 256, 3)


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

ResourceExhaustedError: OOM when allocating tensor with shape[62,512,100,100] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradients/conv2d_3/Conv2D_grad/Conv2DBackpropInput (defined at C:\Users\VergilCrimson\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'gradients/conv2d_3/Conv2D_grad/Conv2DBackpropInput':
  File "C:\Users\VergilCrimson\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\VergilCrimson\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\VergilCrimson\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\VergilCrimson\Anaconda3\lib\site-packages\traitlets\config